<a href="https://colab.research.google.com/github/INVISIBLE-SAM/SemEval-2025-Task-11---Track-A/blob/main/ml_based_method_E5_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# labse hindi

In [1]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/hin.csv")
test_df = pd.read_csv("/content/hin_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       422
     disgust       1.00      1.00      1.00       265
        fear       1.00      1.00      1.00       380
         joy       1.00      1.00      1.00       442
     sadness       1.00      1.00      1.00       449
    surprise       1.00      1.00      1.00       278

   micro avg       1.00      1.00      1.00      2236
   macro avg       1.00      1.00      1.00      2236
weighted avg       1.00      1.00      1.00      2236
 samples avg       0.76      0.76      0.76      2236

Macro F1 Score for Train Set: 1.0

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.81      0.81      0.81        16
     disgust       0.70      0.70      0.70        10
        fear       0.92      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.95      0.98       422
     disgust       1.00      0.86      0.92       265
        fear       1.00      0.91      0.95       380
         joy       1.00      0.98      0.99       442
     sadness       1.00      0.94      0.97       449
    surprise       1.00      0.92      0.96       278

   micro avg       1.00      0.93      0.97      2236
   macro avg       1.00      0.93      0.96      2236
weighted avg       1.00      0.93      0.97      2236
 samples avg       0.73      0.72      0.72      2236

Macro F1 Score for Train Set: 0.9620522251838709

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.06      0.12        16
     disgust       1.00      0.20      0.33        10
        fear       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# Amharic

In [3]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/amh.csv")
test_df = pd.read_csv("/content/amh_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.94      0.89      0.92      1188
     disgust       0.97      0.90      0.93      1268
        fear       1.00      0.27      0.42       109
         joy       1.00      0.83      0.91       549
     sadness       1.00      0.79      0.88       771
    surprise       1.00      0.52      0.69       151

   micro avg       0.97      0.84      0.90      4036
   macro avg       0.98      0.70      0.79      4036
weighted avg       0.97      0.84      0.89      4036
 samples avg       0.72      0.70      0.70      4036

Macro F1 Score for Train Set: 0.7917302619300876

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.66      0.56      0.60       207
     disgust       0.77      0.64      0.70       209
        fear       0.00      0.00      0.00        22
         joy       0.84      0.45      0.59        93
     sadness       0.83      0.31      0.46       127
    surprise       1.00      0.04      0.07        27

   micro avg       0.74      0.48      0.59       685
   macro avg       0.68      0.33      0.40       685
weighted avg       0.74      0.48      0.56       685
 samples avg       0.43      0.42      0.41       685

Macro F1 Score for Test Set: 0.4027476268641191

Summary for SVM:
  Train Macro F1 Score: 0.7917302619300876
  Test Macro F1 Score:  0.4027476268641191

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.59      0.79      0.67      1188
     disgust       0.63      0.78      0.70      1268
        fear       0.28      0.79      0.41       109
         joy       0.51      0.81      0.63       549
     sadness       0.50      0.70      0.58       771
    surprise       0.30      0.83      0.44       151

   micro avg       0.54      0.77      0.63      4036
   macro avg       0.47      0.78      0.57      4036
weighted avg       0.56      0.77      0.64      4036
 samples avg       0.46      0.63      0.51      4036

Macro F1 Score for Train Set: 0.5728842698042392

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.55      0.73      0.63       207
     disgust       0.61      0.77      0.68       209
        fear       0.25      0.59      0.35        22
   

...

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.85      0.82      0.83      1188
     disgust       0.89      0.86      0.88      1268
        fear       1.00      1.00      1.00       109
         joy       1.00      0.99      1.00       549
     sadness       0.94      0.88      0.91       771
    surprise       1.00      1.00      1.00       151

   micro avg       0.91      0.88      0.89      4036
   macro avg       0.95      0.93      0.94      4036
weighted avg       0.91      0.88      0.89      4036
 samples avg       0.73      0.73      0.72      4036

Macro F1 Score for Train Set: 0.9357772883703337

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.59      0.59      0.59       207
     disgust       0.61      0.65      0.63       209
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.99      0.99      1188
     disgust       1.00      0.99      0.99      1268
        fear       1.00      0.04      0.07       109
         joy       1.00      0.77      0.87       549
     sadness       1.00      0.79      0.88       771
    surprise       1.00      0.11      0.19       151

   micro avg       1.00      0.86      0.93      4036
   macro avg       1.00      0.61      0.67      4036
weighted avg       1.00      0.86      0.90      4036
 samples avg       0.75      0.71      0.72      4036

Macro F1 Score for Train Set: 0.6670336382893183

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.71      0.31      0.43       207
     disgust       0.84      0.38      0.53       209
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

anger: 0
disgust: 0
fear: 0
joy: 0
sadness: 0
surprise: 0


# arabic

In [4]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/ary.csv")
test_df = pd.read_csv("/content/ary_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.74      0.85       380
     disgust       1.00      0.44      0.61        66
        fear       1.00      0.67      0.80       119
         joy       1.00      0.82      0.90       302
     sadness       1.00      0.77      0.87       246
    surprise       1.00      0.69      0.82       246

   micro avg       1.00      0.74      0.85      1359
   macro avg       1.00      0.69      0.81      1359
weighted avg       1.00      0.74      0.84      1359
 samples avg       0.56      0.54      0.55      1359

Macro F1 Score for Train Set: 0.8091595475180168

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.89      0.25      0.40        63
     disgust       0.00      0.00      0.00        11
        fear       0.00      0.00      0.00        20
         joy       0.83      0.30      0.44        50
     sadness       0.85      0.41      0.56        41
    surprise       1.00      0.05      0.10        39

   micro avg       0.86      0.22      0.35       224
   macro avg       0.60      0.17      0.25       224
weighted avg       0.77      0.22      0.33       224
 samples avg       0.19      0.17      0.17       224

Macro F1 Score for Test Set: 0.24852937062889682

Summary for SVM:
  Train Macro F1 Score: 0.8091595475180168
  Test Macro F1 Score:  0.24852937062889682

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.52      0.78      0.62

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       380
     disgust       1.00      1.00      1.00        66
        fear       1.00      1.00      1.00       119
         joy       1.00      1.00      1.00       302
     sadness       1.00      1.00      1.00       246
    surprise       1.00      1.00      1.00       246

   micro avg       1.00      1.00      1.00      1359
   macro avg       1.00      1.00      1.00      1359
weighted avg       1.00      1.00      1.00      1359
 samples avg       0.72      0.72      0.72      1359

Macro F1 Score for Train Set: 1.0

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.50      0.43      0.46        63
     disgust       0.25      0.18      0.21        11
        fear       0.38      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.95      0.98       380
     disgust       0.00      0.00      0.00        66
        fear       1.00      0.42      0.59       119
         joy       1.00      0.92      0.96       302
     sadness       1.00      0.66      0.79       246
    surprise       1.00      0.82      0.90       246

   micro avg       1.00      0.77      0.87      1359
   macro avg       0.83      0.63      0.70      1359
weighted avg       0.95      0.77      0.84      1359
 samples avg       0.61      0.58      0.59      1359

Macro F1 Score for Train Set: 0.7033637807748762

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.02      0.03        63
     disgust       0.00      0.00      0.00        11
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# chinese

In [5]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/chn.csv")
test_df = pd.read_csv("/content/chn_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.96      0.98      0.97      1178
     disgust       1.00      0.42      0.59       403
        fear       1.00      0.23      0.37        71
         joy       0.99      0.96      0.98       529
     sadness       1.00      0.66      0.79       354
    surprise       1.00      0.34      0.51       178

   micro avg       0.98      0.79      0.87      2713
   macro avg       0.99      0.60      0.70      2713
weighted avg       0.98      0.79      0.85      2713
 samples avg       0.70      0.64      0.66      2713

Macro F1 Score for Train Set: 0.7020999868455745

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.78      0.89      0.83        92
     disgust       0.00      0.00      0.00        32
        fear       0.00      0.00      0.00         5
         joy       0.94      0.84      0.89        37
     sadness       0.71      0.23      0.34        22
    surprise       0.00      0.00      0.00        17

   micro avg       0.81      0.58      0.67       205
   macro avg       0.41      0.33      0.34       205
weighted avg       0.60      0.58      0.57       205
 samples avg       0.58      0.48      0.51       205

Macro F1 Score for Test Set: 0.34383819692764206

Summary for SVM:
  Train Macro F1 Score: 0.7020999868455745
  Test Macro F1 Score:  0.34383819692764206

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.72      0.84      0.77      1178
     disgust       0.30      0.79      0.44       403
        fear       0.15      0.86      0.25        71
         joy       0.75      0.85      0.80       529
     sadness       0.40      0.79      0.53       354
    surprise       0.25      0.88      0.39       178

   micro avg       0.47      0.83      0.60      2713
   macro avg       0.43      0.83      0.53      2713
weighted avg       0.58      0.83      0.66      2713
 samples avg       0.45      0.64      0.50      2713

Macro F1 Score for Train Set: 0.5300024744167556

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.78      0.88      0.83        92
     disgust       0.28      0.78      0.41        32
        fear       0.14      0.80      0.24         5
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.99      0.99      0.99      1178
     disgust       1.00      0.97      0.98       403
        fear       1.00      1.00      1.00        71
         joy       1.00      1.00      1.00       529
     sadness       1.00      1.00      1.00       354
    surprise       1.00      1.00      1.00       178

   micro avg       1.00      0.99      1.00      2713
   macro avg       1.00      0.99      1.00      2713
weighted avg       1.00      0.99      1.00      2713
 samples avg       0.77      0.77      0.77      2713

Macro F1 Score for Train Set: 0.9964907291386017

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.69      0.71      0.70        92
     disgust       0.36      0.44      0.39        32
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.99      1.00      1.00      1178
     disgust       1.00      0.44      0.61       403
        fear       0.00      0.00      0.00        71
         joy       1.00      0.95      0.97       529
     sadness       1.00      0.50      0.66       354
    surprise       0.00      0.00      0.00       178

   micro avg       1.00      0.75      0.86      2713
   macro avg       0.67      0.48      0.54      2713
weighted avg       0.91      0.75      0.80      2713
 samples avg       0.68      0.60      0.63      2713

Macro F1 Score for Train Set: 0.5409566762115132

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.80      0.85      0.83        92
     disgust       0.00      0.00      0.00        32
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# german

In [6]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/deu.csv")
test_df = pd.read_csv("/content/deu_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.97      0.93      0.95       768
     disgust       0.96      0.92      0.94       832
        fear       1.00      0.48      0.65       239
         joy       0.99      0.86      0.92       541
     sadness       1.00      0.80      0.89       516
    surprise       1.00      0.39      0.56       159

   micro avg       0.98      0.83      0.90      3055
   macro avg       0.99      0.73      0.82      3055
weighted avg       0.98      0.83      0.89      3055
 samples avg       0.65      0.62      0.63      3055

Macro F1 Score for Train Set: 0.8177280794444788

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.74      0.66      0.70        61
     disgust       0.71      0.59      0.64        61
        fear       0.00      0.00      0.00        20
         joy       0.95      0.47      0.63        40
     sadness       0.94      0.31      0.47        48
    surprise       0.00      0.00      0.00        15

   micro avg       0.78      0.45      0.57       245
   macro avg       0.56      0.34      0.41       245
weighted avg       0.70      0.45      0.53       245
 samples avg       0.37      0.34      0.34       245

Macro F1 Score for Test Set: 0.4067654416839199

Summary for SVM:
  Train Macro F1 Score: 0.8177280794444788
  Test Macro F1 Score:  0.4067654416839199

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.72      0.80      0.76       768
     disgust       0.68      0.74      0.71       832
        fear       0.32      0.81      0.46       239
         joy       0.55      0.80      0.65       541
     sadness       0.52      0.75      0.61       516
    surprise       0.41      0.85      0.55       159

   micro avg       0.56      0.78      0.65      3055
   macro avg       0.53      0.79      0.62      3055
weighted avg       0.60      0.78      0.67      3055
 samples avg       0.48      0.58      0.51      3055

Macro F1 Score for Train Set: 0.6236856738458265

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.68      0.74      0.71        61
     disgust       0.59      0.67      0.63        61
        fear       0.18      0.45      0.26        20
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       768
     disgust       1.00      0.99      0.99       832
        fear       1.00      1.00      1.00       239
         joy       1.00      1.00      1.00       541
     sadness       1.00      1.00      1.00       516
    surprise       1.00      1.00      1.00       159

   micro avg       1.00      1.00      1.00      3055
   macro avg       1.00      1.00      1.00      3055
weighted avg       1.00      1.00      1.00      3055
 samples avg       0.75      0.75      0.75      3055

Macro F1 Score for Train Set: 0.9986261555419622

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.70      0.61      0.65        61
     disgust       0.58      0.66      0.62        61
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.97      0.98       768
     disgust       1.00      0.96      0.98       832
        fear       1.00      0.05      0.09       239
         joy       1.00      0.86      0.93       541
     sadness       1.00      0.77      0.87       516
    surprise       0.00      0.00      0.00       159

   micro avg       1.00      0.79      0.88      3055
   macro avg       0.83      0.60      0.64      3055
weighted avg       0.95      0.79      0.83      3055
 samples avg       0.64      0.59      0.61      3055

Macro F1 Score for Train Set: 0.6412383263418913

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.78      0.41      0.54        61
     disgust       0.75      0.39      0.52        61
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# hausa

In [7]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/hau.csv")
test_df = pd.read_csv("/content/hau_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.74      0.85       408
     disgust       1.00      0.75      0.86       329
        fear       0.99      0.89      0.94       327
         joy       0.99      0.76      0.86       320
     sadness       0.96      0.88      0.92       647
    surprise       1.00      0.75      0.86       349

   micro avg       0.99      0.81      0.89      2380
   macro avg       0.99      0.80      0.88      2380
weighted avg       0.99      0.81      0.88      2380
 samples avg       0.74      0.71      0.72      2380

Macro F1 Score for Train Set: 0.880511802269987

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.70      0.21      0.32        67
     disgust       0.86      0.56      0.68        55
        fear       0.88      0.53      0.66        53
         joy       0.89      0.45      0.60        53
     sadness       0.82      0.51      0.63       109
    surprise       0.95      0.32      0.47        57

   micro avg       0.85      0.43      0.57       394
   macro avg       0.85      0.43      0.56       394
weighted avg       0.84      0.43      0.56       394
 samples avg       0.42      0.40      0.40       394

Macro F1 Score for Test Set: 0.5614056438830756

Summary for SVM:
  Train Macro F1 Score: 0.880511802269987
  Test Macro F1 Score:  0.5614056438830756

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

              precision    recall  f1-score   support

       anger       0.44      0.73      0.54       408
     disgust       0.53      0.76      0.62       329
        fear       0.49      0.73      0.59       327
         joy       0.51      0.74      0.60       320
     sadness       0.58      0.76      0.66       647
    surprise       0.39      0.69      0.50       349

   micro avg       0.49      0.74      0.59      2380
   macro avg       0.49      0.74      0.59      2380
weighted avg       0.50      0.74      0.59      2380
 samples avg       0.50      0.65      0.54      2380

Macro F1 Score for Train Set: 0.5860181093750018

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.42      0.66      0.51        67
     disgust       0.50      0.76      0.60        55
        fear       0.51      0.74      0.60        53
         joy       0.41      0.60      0.48        53
    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.75      0.86       408
     disgust       1.00      0.76      0.87       329
        fear       1.00      0.82      0.90       327
         joy       1.00      0.71      0.83       320
     sadness       1.00      0.95      0.97       647
    surprise       1.00      0.59      0.74       349

   micro avg       1.00      0.79      0.88      2380
   macro avg       1.00      0.76      0.86      2380
weighted avg       1.00      0.79      0.88      2380
 samples avg       0.72      0.69      0.70      2380

Macro F1 Score for Train Set: 0.8616289787433558

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        67
     disgust       1.00      0.24      0.38        55
        fear       0.88     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

#marathi

In [8]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/mar.csv")
test_df = pd.read_csv("/content/mar_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.82      0.90       350
     disgust       1.00      0.85      0.92       299
        fear       1.00      0.92      0.96       382
         joy       1.00      0.93      0.96       461
     sadness       0.99      0.87      0.93       431
    surprise       1.00      0.94      0.97       311

   micro avg       1.00      0.89      0.94      2234
   macro avg       1.00      0.89      0.94      2234
weighted avg       1.00      0.89      0.94      2234
 samples avg       0.78      0.76      0.76      2234

Macro F1 Score for Train Set: 0.9392631884249765

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.57      0.73        14
     disgust       1.00      0.73      0.84        11
        fear       1.00      0.87      0.93        15
         joy       1.00      0.89      0.94        19
     sadness       0.93      0.76      0.84        17
    surprise       1.00      0.92      0.96        12

   micro avg       0.99      0.80      0.88        88
   macro avg       0.99      0.79      0.87        88
weighted avg       0.99      0.80      0.88        88
 samples avg       0.69      0.67      0.67        88

Macro F1 Score for Test Set: 0.8729375466660474

Summary for SVM:
  Train Macro F1 Score: 0.9392631884249765
  Test Macro F1 Score:  0.8729375466660474

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.39      0.81      0.53       350
     disgust       0.40      0.82      0.54       299
        fear       0.69      0.83      0.75       382
         joy       0.67      0.92      0.78       461
     sadness       0.41      0.81      0.55       431
    surprise       0.78      0.86      0.82       311

   micro avg       0.52      0.85      0.64      2234
   macro avg       0.56      0.84      0.66      2234
weighted avg       0.56      0.85      0.66      2234
 samples avg       0.52      0.72      0.57      2234

Macro F1 Score for Train Set: 0.6596436233523213

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.41      0.86      0.56        14
     disgust       0.36      0.73      0.48        11
        fear       0.72      0.87      0.79        15
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       350
     disgust       1.00      1.00      1.00       299
        fear       1.00      1.00      1.00       382
         joy       1.00      1.00      1.00       461
     sadness       1.00      1.00      1.00       431
    surprise       1.00      1.00      1.00       311

   micro avg       1.00      1.00      1.00      2234
   macro avg       1.00      1.00      1.00      2234
weighted avg       1.00      1.00      1.00      2234
 samples avg       0.83      0.83      0.83      2234

Macro F1 Score for Train Set: 1.0

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.73      0.79      0.76        14
     disgust       1.00      0.91      0.95        11
        fear       1.00      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.94      0.97       350
     disgust       1.00      0.85      0.92       299
        fear       1.00      0.93      0.96       382
         joy       1.00      0.98      0.99       461
     sadness       1.00      0.90      0.95       431
    surprise       1.00      0.92      0.96       311

   micro avg       1.00      0.92      0.96      2234
   macro avg       1.00      0.92      0.96      2234
weighted avg       1.00      0.92      0.96      2234
 samples avg       0.79      0.77      0.78      2234

Macro F1 Score for Train Set: 0.9577887100148245

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.29      0.44        14
     disgust       1.00      0.45      0.62        11
        fear       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# oromo

In [9]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/orm.csv")
test_df = pd.read_csv("/content/orm_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.98      0.68      0.81       646
     disgust       0.99      0.67      0.80       557
        fear       1.00      0.05      0.09       123
         joy       0.98      0.91      0.95      1091
     sadness       1.00      0.42      0.59       298
    surprise       1.00      0.57      0.72       129

   micro avg       0.99      0.71      0.82      2844
   macro avg       0.99      0.55      0.66      2844
weighted avg       0.99      0.71      0.80      2844
 samples avg       0.57      0.55      0.56      2844

Macro F1 Score for Train Set: 0.6588929298423242

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.52      0.22      0.31       108
     disgust       0.79      0.20      0.32        94
        fear       0.00      0.00      0.00        21
         joy       0.83      0.66      0.74       183
     sadness       1.00      0.02      0.04        52
    surprise       0.67      0.15      0.24        27

   micro avg       0.76      0.35      0.48       485
   macro avg       0.64      0.21      0.28       485
weighted avg       0.73      0.35      0.43       485
 samples avg       0.29      0.28      0.28       485

Macro F1 Score for Test Set: 0.2752811965871705

Summary for SVM:
  Train Macro F1 Score: 0.6588929298423242
  Test Macro F1 Score:  0.2752811965871705

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_


Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.36      0.79      0.50       646
     disgust       0.41      0.78      0.53       557
        fear       0.11      0.77      0.20       123
         joy       0.63      0.73      0.68      1091
     sadness       0.24      0.74      0.36       298
    surprise       0.24      0.71      0.36       129

   micro avg       0.37      0.76      0.49      2844
   macro avg       0.33      0.76      0.44      2844
weighted avg       0.44      0.76      0.54      2844
 samples avg       0.39      0.58      0.44      2844

Macro F1 Score for Train Set: 0.43725256017011377

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.34      0.79      0.47       108
     disgust       0.35      0.66      0.45        94
        fear       0.08      0.57      0.14        21
  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.93      0.86      0.89       646
     disgust       0.95      0.89      0.92       557
        fear       1.00      1.00      1.00       123
         joy       0.98      0.95      0.96      1091
     sadness       1.00      0.99      1.00       298
    surprise       1.00      1.00      1.00       129

   micro avg       0.97      0.93      0.95      2844
   macro avg       0.98      0.95      0.96      2844
weighted avg       0.96      0.93      0.95      2844
 samples avg       0.71      0.71      0.71      2844

Macro F1 Score for Train Set: 0.9622605375350397

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.40      0.45      0.43       108
     disgust       0.56      0.57      0.57        94
        fear    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.82      0.90       646
     disgust       1.00      0.43      0.60       557
        fear       0.00      0.00      0.00       123
         joy       1.00      0.95      0.97      1091
     sadness       1.00      0.15      0.27       298
    surprise       1.00      0.16      0.27       129

   micro avg       1.00      0.66      0.79      2844
   macro avg       0.83      0.42      0.50      2844
weighted avg       0.96      0.66      0.74      2844
 samples avg       0.53      0.51      0.51      2844

Macro F1 Score for Train Set: 0.5014431883865434

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.50      0.01      0.02       108
     disgust       0.00      0.00      0.00        94
        fear       0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

#romanian

In [10]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/ron.csv")
test_df = pd.read_csv("/content/ron_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.90      0.95       210
     disgust       1.00      0.97      0.98       297
        fear       1.00      0.99      1.00       405
         joy       1.00      0.99      1.00       462
     sadness       0.99      0.98      0.99       379
    surprise       1.00      0.87      0.93       457

   micro avg       1.00      0.95      0.98      2210
   macro avg       1.00      0.95      0.97      2210
weighted avg       1.00      0.95      0.97      2210
 samples avg       0.91      0.88      0.89      2210

Macro F1 Score for Train Set: 0.9736568172663679

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.17      0.29        24
     disgust       0.88      0.48      0.62        29
        fear       0.78      0.78      0.78        41
         joy       0.98      1.00      0.99        45
     sadness       0.82      0.74      0.78        38
    surprise       0.47      0.21      0.29        38

   micro avg       0.83      0.61      0.70       215
   macro avg       0.82      0.56      0.62       215
weighted avg       0.81      0.61      0.66       215
 samples avg       0.67      0.58      0.60       215

Macro F1 Score for Test Set: 0.6243536950854024

Summary for SVM:
  Train Macro F1 Score: 0.9736568172663679
  Test Macro F1 Score:  0.6243536950854024

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.39      0.93      0.55  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       210
     disgust       1.00      1.00      1.00       297
        fear       1.00      1.00      1.00       405
         joy       1.00      1.00      1.00       462
     sadness       1.00      1.00      1.00       379
    surprise       1.00      1.00      1.00       457

   micro avg       1.00      1.00      1.00      2210
   macro avg       1.00      1.00      1.00      2210
weighted avg       1.00      1.00      1.00      2210
 samples avg       0.91      0.91      0.91      2210

Macro F1 Score for Train Set: 1.0

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.58      0.62        24
     disgust       0.69      0.69      0.69        29
        fear       0.71      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.97      0.99       210
     disgust       1.00      0.99      1.00       297
        fear       1.00      1.00      1.00       405
         joy       1.00      1.00      1.00       462
     sadness       1.00      0.99      1.00       379
    surprise       1.00      1.00      1.00       457

   micro avg       1.00      1.00      1.00      2210
   macro avg       1.00      0.99      1.00      2210
weighted avg       1.00      1.00      1.00      2210
 samples avg       0.91      0.91      0.91      2210

Macro F1 Score for Train Set: 0.9965805608921552

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        24
     disgust       0.50      0.03      0.06        29
        fear       0.83     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# rusian

In [11]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/rus.csv")
test_df = pd.read_csv("/content/rus_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.99      0.95      0.97       543
     disgust       1.00      0.89      0.94       273
        fear       1.00      0.95      0.98       328
         joy       0.99      0.97      0.98       555
     sadness       1.00      0.90      0.95       421
    surprise       1.00      0.90      0.95       355

   micro avg       1.00      0.93      0.96      2475
   macro avg       1.00      0.93      0.96      2475
weighted avg       1.00      0.93      0.96      2475
 samples avg       0.78      0.76      0.77      2475

Macro F1 Score for Train Set: 0.9597174444289424

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.87      0.70      0.78        47
     disgust       1.00      0.62      0.76        26
        fear       0.95      0.86      0.90        21
         joy       0.82      0.68      0.74        34
     sadness       0.89      0.64      0.75        39
    surprise       0.88      0.54      0.67        26

   micro avg       0.89      0.67      0.76       193
   macro avg       0.90      0.67      0.77       193
weighted avg       0.89      0.67      0.76       193
 samples avg       0.63      0.61      0.61       193

Macro F1 Score for Test Set: 0.7655410262323513

Summary for SVM:
  Train Macro F1 Score: 0.9597174444289424
  Test Macro F1 Score:  0.7655410262323513

Training Naive Bayes...
Evaluation on Train Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.63      0.89      0.74       543
     disgust       0.52      0.90      0.66       273
        fear       0.81      0.87      0.84       328
         joy       0.79      0.89      0.84       555
     sadness       0.63      0.88      0.73       421
    surprise       0.58      0.82      0.68       355

   micro avg       0.66      0.88      0.75      2475
   macro avg       0.66      0.87      0.75      2475
weighted avg       0.67      0.88      0.76      2475
 samples avg       0.58      0.72      0.62      2475

Macro F1 Score for Train Set: 0.7485397694585038

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.66      0.91      0.77        47
     disgust       0.70      0.81      0.75        26
        fear       0.67      0.95      0.78        21
   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       543
     disgust       1.00      1.00      1.00       273
        fear       1.00      1.00      1.00       328
         joy       1.00      1.00      1.00       555
     sadness       1.00      1.00      1.00       421
    surprise       1.00      1.00      1.00       355

   micro avg       1.00      1.00      1.00      2475
   macro avg       1.00      1.00      1.00      2475
weighted avg       1.00      1.00      1.00      2475
 samples avg       0.81      0.81      0.81      2475

Macro F1 Score for Train Set: 1.0

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.71      0.79      0.75        47
     disgust       0.80      0.62      0.70        26
        fear       0.83      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.98      0.99       543
     disgust       1.00      0.77      0.87       273
        fear       1.00      0.90      0.95       328
         joy       1.00      0.98      0.99       555
     sadness       1.00      0.93      0.96       421
    surprise       1.00      0.76      0.86       355

   micro avg       1.00      0.90      0.95      2475
   macro avg       1.00      0.89      0.94      2475
weighted avg       1.00      0.90      0.95      2475
 samples avg       0.76      0.74      0.75      2475

Macro F1 Score for Train Set: 0.936800556234639

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.26      0.41        47
     disgust       0.00      0.00      0.00        26
        fear       1.00      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

# spanish

In [12]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/esp.csv")
test_df = pd.read_csv("/content/esp_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.98      0.93      0.96       492
     disgust       0.98      0.97      0.97       654
        fear       0.98      0.94      0.96       317
         joy       0.99      0.97      0.98       642
     sadness       1.00      0.91      0.95       309
    surprise       1.00      0.82      0.90       421

   micro avg       0.99      0.93      0.96      2835
   macro avg       0.99      0.92      0.95      2835
weighted avg       0.99      0.93      0.96      2835
 samples avg       0.97      0.95      0.95      2835

Macro F1 Score for Train Set: 0.9541033216494447

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.63      0.65      0.64        34
     disgust       0.85      0.72      0.78        65
        fear       1.00      0.75      0.86        32
         joy       0.88      0.83      0.86        54
     sadness       1.00      0.68      0.81        31
    surprise       0.87      0.33      0.47        40

   micro avg       0.86      0.67      0.75       256
   macro avg       0.87      0.66      0.74       256
weighted avg       0.87      0.67      0.74       256
 samples avg       0.75      0.72      0.71       256

Macro F1 Score for Test Set: 0.7359532979098197

Summary for SVM:
  Train Macro F1 Score: 0.9541033216494447
  Test Macro F1 Score:  0.7359532979098197

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.69      0.86      0.76  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training Logistic Regression...
Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       492
     disgust       1.00      1.00      1.00       654
        fear       1.00      1.00      1.00       317
         joy       1.00      1.00      1.00       642
     sadness       1.00      1.00      1.00       309
    surprise       1.00      1.00      1.00       421

   micro avg       1.00      1.00      1.00      2835
   macro avg       1.00      1.00      1.00      2835
weighted avg       1.00      1.00      1.00      2835
 samples avg       1.00      1.00      1.00      2835

Macro F1 Score for Train Set: 1.0

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.49      0.65      0.56        34
     disgust       0.77      0.75      0.76        65

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.98      0.99       492
     disgust       1.00      1.00      1.00       654
        fear       1.00      0.95      0.97       317
         joy       1.00      1.00      1.00       642
     sadness       1.00      0.89      0.94       309
    surprise       1.00      0.85      0.92       421

   micro avg       1.00      0.95      0.98      2835
   macro avg       1.00      0.94      0.97      2835
weighted avg       1.00      0.95      0.98      2835
 samples avg       0.98      0.96      0.97      2835

Macro F1 Score for Train Set: 0.9701591683481902

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.24      0.35        34
     disgust       0.87      0.42      0.56        65
        fear       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# ukranian

In [13]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/ukr.csv")
test_df = pd.read_csv("/content/ukr_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the emotion label columns
label_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.26      0.41        98
     disgust       1.00      0.28      0.44        86
        fear       1.00      0.92      0.96       172
         joy       0.99      0.87      0.93       412
     sadness       1.00      0.79      0.88       333
    surprise       1.00      0.69      0.82       196

   micro avg       1.00      0.74      0.85      1297
   macro avg       1.00      0.63      0.74      1297
weighted avg       1.00      0.74      0.83      1297
 samples avg       0.39      0.38      0.38      1297

Macro F1 Score for Train Set: 0.7378061431426016

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        12
     disgust       0.00      0.00      0.00         9
        fear       1.00      0.65      0.79        17
         joy       0.67      0.44      0.53        41
     sadness       0.79      0.32      0.46        34
    surprise       0.75      0.16      0.26        19

   micro avg       0.77      0.33      0.46       132
   macro avg       0.53      0.26      0.34       132
weighted avg       0.65      0.33      0.42       132
 samples avg       0.17      0.17      0.17       132

Macro F1 Score for Test Set: 0.3390548248284821

Summary for SVM:
  Train Macro F1 Score: 0.7378061431426016
  Test Macro F1 Score:  0.3390548248284821

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

       anger       0.37      0.94      0.53        98
     disgust       0.28      0.90      0.43        86
        fear       0.59      0.91      0.72       172
         joy       0.61      0.83      0.71       412
     sadness       0.48      0.83      0.61       333
    surprise       0.41      0.82      0.55       196

   micro avg       0.48      0.85      0.61      1297
   macro avg       0.46      0.87      0.59      1297
weighted avg       0.51      0.85      0.63      1297
 samples avg       0.36      0.43      0.38      1297

Macro F1 Score for Train Set: 0.5889125505626257

Evaluation on Test Set:

Model: Naive Bayes - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.40      0.50      0.44        12
     disgust       0.13      0.33      0.19         9
        fear       0.56      0.88      0.68        17
         joy       0.44      0.68      0.53        41
    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00        98
     disgust       1.00      1.00      1.00        86
        fear       1.00      1.00      1.00       172
         joy       1.00      1.00      1.00       412
     sadness       1.00      1.00      1.00       333
    surprise       1.00      1.00      1.00       196

   micro avg       1.00      1.00      1.00      1297
   macro avg       1.00      1.00      1.00      1297
weighted avg       1.00      1.00      1.00      1297
 samples avg       0.50      0.50      0.50      1297

Macro F1 Score for Train Set: 1.0

Evaluation on Test Set:

Model: Logistic Regression - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.31      0.33      0.32        12
     disgust       0.20      0.11      0.14         9
        fear       0.88      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.01      0.02        98
     disgust       0.00      0.00      0.00        86
        fear       1.00      0.74      0.85       172
         joy       1.00      0.85      0.92       412
     sadness       1.00      0.70      0.82       333
    surprise       1.00      0.35      0.52       196

   micro avg       1.00      0.60      0.75      1297
   macro avg       0.83      0.44      0.52      1297
weighted avg       0.93      0.60      0.69      1297
 samples avg       0.31      0.31      0.31      1297

Macro F1 Score for Train Set: 0.520787661961584

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        12
     disgust       0.00      0.00      0.00         9
        fear       0.00      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# english

In [14]:
import pandas as pd
import numpy as np

# Import classifiers and wrappers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Evaluation metrics
from sklearn.metrics import classification_report, f1_score

# For generating text embeddings
from sentence_transformers import SentenceTransformer

# For scaling the embeddings
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 1. Load Data and Generate Embeddings
# -------------------------------

# Load datasets
train_df = pd.read_csv("/content/esp.csv")
test_df = pd.read_csv("/content/esp_dev.csv")

# Load the LaBSE model for generating embeddings
labse_model = SentenceTransformer('intfloat/multilingual-e5-large')

def generate_embeddings(texts):
    """
    Given a list or array of texts, generate and return their normalized embeddings.
    """
    return np.array(labse_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# Get text data
train_texts = train_df['text'].values
test_texts = test_df['text'].values

# Generate embeddings for train and test sets
train_embeddings = generate_embeddings(train_texts)
test_embeddings = generate_embeddings(test_texts)

# -------------------------------
# 2. Prepare Labels and Scale Features
# -------------------------------

# Define the expected emotion labels.
expected_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

# Create label_columns by including only those labels that exist in both datasets.
label_columns = [label for label in expected_labels if label in train_df.columns and label in test_df.columns]

# Optionally, print which labels will be used.
print("Using emotion labels:", label_columns)

# Extract label arrays
train_labels = train_df[label_columns].values
test_labels = test_df[label_columns].values

# Scale the embeddings (feature standardization)
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

# -------------------------------
# 3. Define, Train, and Evaluate Models
# -------------------------------

# Define the four models. For Random Forest we add some regularization
# by limiting the maximum tree depth and controlling the minimum samples needed for splits.
models = {
    "SVM": MultiOutputClassifier(SVC(kernel='rbf', probability=True, random_state=42)),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "Logistic Regression": MultiOutputClassifier(
        LogisticRegression(max_iter=1000, random_state=42)
    ),
    "Random Forest": MultiOutputClassifier(
        RandomForestClassifier(max_depth=10, min_samples_split=5, random_state=42)
    )
}

def evaluate_model(model, embeddings, labels, dataset_name, model_name):
    """
    Predict labels using the given model and embeddings, then print the classification report
    and macro F1 score.
    """
    predictions = model.predict(embeddings)
    print(f"\nModel: {model_name} - {dataset_name}")
    print("Classification Report:")
    print(classification_report(labels, predictions, target_names=label_columns))

    macro_f1 = f1_score(labels, predictions, average='macro')
    print(f"Macro F1 Score for {dataset_name}: {macro_f1}\n")
    return macro_f1

# Train and evaluate each model on both the training and testing sets.
for model_name, clf in models.items():
    print("==============================================")
    print(f"Training {model_name}...")
    clf.fit(train_embeddings_scaled, train_labels)

    # Evaluate on the Training Set
    print("Evaluation on Train Set:")
    train_macro_f1 = evaluate_model(clf, train_embeddings_scaled, train_labels, "Train Set", model_name)

    # Evaluate on the Testing Set
    print("Evaluation on Test Set:")
    test_macro_f1 = evaluate_model(clf, test_embeddings_scaled, test_labels, "Test Set", model_name)

    print(f"Summary for {model_name}:")
    print(f"  Train Macro F1 Score: {train_macro_f1}")
    print(f"  Test Macro F1 Score:  {test_macro_f1}")
    print("==============================================\n")

# -------------------------------
# 4. Predict Emotions for New Text
# -------------------------------

def predict_emotions(text, model, scaler):
    """
    Given a new text string, generate its embedding, scale it, and predict emotion labels.
    Returns a dictionary mapping each available emotion to its predicted label (0 or 1).
    """
    # Generate embedding for the new text
    embedding = generate_embeddings([text])
    embedding_scaled = scaler.transform(embedding)

    # Get predictions
    predictions = model.predict(embedding_scaled)

    # Map each label to its corresponding emotion
    emotion_preds = {emotion: int(pred) for emotion, pred in zip(label_columns, predictions[0])}
    return emotion_preds

# Example usage for prediction using the SVM model (you can choose any of the trained models)
example_text = "यह एक उदाहरण वाक्य है"
print("Example prediction using SVM:")
svm_model = models["SVM"]
example_prediction = predict_emotions(example_text, svm_model, scaler)
for emotion, pred in example_prediction.items():
    print(f"{emotion}: {pred}")


Using emotion labels: ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
Training SVM...
Evaluation on Train Set:

Model: SVM - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.98      0.93      0.96       492
     disgust       0.98      0.97      0.97       654
        fear       0.98      0.94      0.96       317
         joy       0.99      0.97      0.98       642
     sadness       1.00      0.91      0.95       309
    surprise       1.00      0.82      0.90       421

   micro avg       0.99      0.93      0.96      2835
   macro avg       0.99      0.92      0.95      2835
weighted avg       0.99      0.93      0.96      2835
 samples avg       0.97      0.95      0.95      2835

Macro F1 Score for Train Set: 0.9541033216494447

Evaluation on Test Set:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: SVM - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.63      0.65      0.64        34
     disgust       0.85      0.72      0.78        65
        fear       1.00      0.75      0.86        32
         joy       0.88      0.83      0.86        54
     sadness       1.00      0.68      0.81        31
    surprise       0.87      0.33      0.47        40

   micro avg       0.86      0.67      0.75       256
   macro avg       0.87      0.66      0.74       256
weighted avg       0.87      0.67      0.74       256
 samples avg       0.75      0.72      0.71       256

Macro F1 Score for Test Set: 0.7359532979098197

Summary for SVM:
  Train Macro F1 Score: 0.9541033216494447
  Test Macro F1 Score:  0.7359532979098197

Training Naive Bayes...
Evaluation on Train Set:

Model: Naive Bayes - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.69      0.86      0.76  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       anger       0.53      0.82      0.64        34
     disgust       0.78      0.75      0.77        65
        fear       0.90      0.81      0.85        32
         joy       0.80      0.81      0.81        54
     sadness       0.77      0.87      0.82        31
    surprise       0.56      0.62      0.59        40

   micro avg       0.71      0.78      0.74       256
   macro avg       0.72      0.78      0.75       256
weighted avg       0.73      0.78      0.75       256
 samples avg       0.72      0.82      0.73       256

Macro F1 Score for Test Set: 0.7459197898589555

Summary for Naive Bayes:
  Train Macro F1 Score: 0.8000074292389535
  Test Macro F1 Score:  0.7459197898589555

Training Logistic Regression...
Evaluation on Train Set:

Model: Logistic Regression - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00       492
     disgust 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation on Train Set:

Model: Random Forest - Train Set
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.98      0.99       492
     disgust       1.00      1.00      1.00       654
        fear       1.00      0.95      0.97       317
         joy       1.00      1.00      1.00       642
     sadness       1.00      0.89      0.94       309
    surprise       1.00      0.85      0.92       421

   micro avg       1.00      0.95      0.98      2835
   macro avg       1.00      0.94      0.97      2835
weighted avg       1.00      0.95      0.98      2835
 samples avg       0.98      0.96      0.97      2835

Macro F1 Score for Train Set: 0.9701591683481902

Evaluation on Test Set:

Model: Random Forest - Test Set
Classification Report:
              precision    recall  f1-score   support

       anger       0.67      0.24      0.35        34
     disgust       0.87      0.42      0.56        65
        fear       1.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
